In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

# Selección de modelo
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

Previamente, tuvimos nuestro modelo de regresión múltiple, el cual mostró un rendimiento muy bueno. Ahora surge la pregunta: ¿Podemos obtener un modelo igual o similar utilizando menos atributos? Nuestro análisis estadístico inicial sugiere que algunos atributos podrían estar de más.

Recordemos que un modelo más simple, con menos atributos, es preferible porque reduce la complejidad del modelo, mejora la interpretabilidad, disminuye el riesgo de sobreajuste y puede resultar en un modelo más eficiente.

Por lo tanto, realizaremos una selección de modelo mediante el método de eliminación hacia atrás, utilizando el **criterio de información de Akaike (AIC)** para determinar si un atributo aporta significativamente al modelo.

Comencemos creando una función que nos permita calcular el AIC usando la fórmula que desarrollamos previamente en la parte teórica:

In [2]:
def aic_criterion(y: np.ndarray, y_pred: np.ndarray, num_attributes: int) -> float:
    """
    Calcula el Criterio de Información de Akaike (AIC) para un modelo de regresión lineal.

    Este criterio se utiliza para la selección de modelos, penalizando la complejidad del modelo 
    (el número de parámetros) y favoreciendo a los modelos que tienen un buen ajuste a los datos 
    sin sobreajustar.

    Args:
        y (numpy.ndarray): El vector de valores reales de la variable dependiente (target).
        y_pred (numpy.ndarray): El vector de valores predichos por el modelo de regresión.
        num_attributes (int): El número de atributos (características) utilizados en el modelo.

    Returns:
        float: El valor del Criterio de Información de Akaike (AIC) para el modelo.
    """
    # Agregamos uno porque hay que incorporar a la ordenada al origen
    d = num_attributes + 1
    N = y.shape[0]

    # Calculamos los residuos al cuadrado
    residuals = y - y_pred
    Se = np.sum(residuals**2)

    # Calculamos la estimación del logaritmo de maxima similitud de la regresión lineal
    log_lik = np.log(2*np.pi) + np.log(Se/N) + 1
    log_lik *= -N/2

    #Calculamos ambos criterios
    aic = 2*d - 2*log_lik 
   
    return aic

A continuación, carguemos el dataset completo:

In [3]:
dataset = pd.read_csv("datasets/50_Startups.csv") 

Dado que necesitamos entrenar varios modelos, primero realizamos la separación del dataset y el preprocesamiento de los datos:

In [4]:
from sklearn.model_selection import train_test_split

X = dataset.drop(columns='Profit')
y = dataset["Profit"]

# Usamos random_state=42 para asegurar que el split del dataset sea consistente con el anterior notebook
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Ahora, armemos el preprocesador que aplicará el `One-Hot Encoding` a las variables categóricas y la estandarización a las variables numéricas:

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_columns = ['State']
numerical_columns = ['R&D Spend', 'Administration', 'Marketing Spend']

# Creamos el preprocesamiento para las columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_columns), 
        ('num', StandardScaler(), numerical_columns)   
    ]
)

X_train_processed = preprocessor.fit_transform(X_train)

Para facilitar el siguiente trabajo, transformamos las salidas del preprocesador en un dataframe:

In [6]:
X_train_processed = pd.DataFrame(X_train_processed, columns=preprocessor.get_feature_names_out())

X_train_processed.head()

,cat__State_Florida,cat__State_New York,num__R&D Spend,num__Administration,num__Marketing Spend
0,0.0,0.0,1.309487,0.946637,-0.855429
1,1.0,0.0,-0.966618,-1.507490,-0.536649
2,1.0,0.0,-1.533444,-0.285870,0.613821
3,0.0,0.0,-1.561500,0.484311,-1.963166
4,0.0,1.0,0.880982,-0.018786,0.307319


Las columnas del array `X_train_processed` es:
- 0: `'cat__State_Florida'`
- 1: `'cat__State_New York'`
- 2: `'num__R&D Spend'`
- 3: `'num__Administration'`
- 4: `'num__Marketing Spend'`

Ahora, vamos a armar el modelo utilizando todos los atributos y calcular el AIC:

In [7]:
from sklearn.linear_model import LinearRegression

regresion = LinearRegression()
regresion.fit(X_train_processed, y_train)
y_pred = regresion.predict(X_train_processed)

aic = aic_criterion(y_train, y_pred, X_train_processed.shape[1])

print(f"AIC inicial es {np.round(aic)}")

AIC inicial es 749.0


A continuación, creamos una función que, dada una selección de columnas, entrena un modelo y obtiene el AIC. Esto nos ayudará a ir avanzando paso a paso en la eliminación de atributos:

In [8]:
def train_reg_model(X: pd.DataFrame, y: np.ndarray, columns: list) -> float:
    """
    Entrena un modelo de regresión lineal y calcula el Criterio de Información de Akaike (AIC) para el modelo.

    Esta función realiza lo siguiente:
    1. Selecciona las columnas especificadas del conjunto de datos de características.
    2. Entrena un modelo de regresión lineal utilizando las características seleccionadas y la variable dependiente.
    3. Realiza las predicciones con el modelo entrenado.
    4. Calcula el AIC utilizando las predicciones y las características del modelo.

    Args:
        X (pandas.DataFrame): El DataFrame de características (atributos) de las observaciones.
        y (numpy.ndarray): El vector de valores reales de la variable dependiente (target).
        columns (list): Una lista de las columnas que se deben seleccionar de `X` para entrenar el modelo.

    Returns:
        float: El valor del Criterio de Información de Akaike (AIC) para el modelo entrenado.
    """
    X_clear = X.loc[:, columns].copy()
    
    model = LinearRegression()
    model.fit(X_clear, y)
    y_pred = model.predict(X_clear)

    return aic_criterion(y, y_pred, X_clear.shape[1])

Con esto, estamos listos para realizar la eliminación de atributos hacia atrás y evaluar el rendimiento del modelo a medida que vamos simplificándolo.

### Eliminación hacia atrás

Ya hemos creado el primer modelo con todos los atributos, y obtuvimos un AIC de `749`. Ahora, entrenaremos varios modelos eliminando uno de los cinco atributos en cada paso, y observaremos cómo cambia el valor del AIC. Aplicaremos un enfoque greedy, eligiendo siempre eliminar el atributo que más reduzca el AIC. Continuaremos este proceso hasta que eliminar un atributo no mejore más el AIC.

A continuación, entrenamos 5 modelos diferentes, eliminando un atributo en cada caso:

In [9]:
# cat__State_Florida
col = ['cat__State_New York', 'num__R&D Spend',
       'num__Administration', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[0]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# cat__State_New York
col = ['cat__State_Florida', 'num__R&D Spend',
       'num__Administration', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[1]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__R&D Spend
col = ['cat__State_Florida', 'cat__State_New York',
       'num__Administration', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[2]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Administration
col = ['cat__State_Florida', 'cat__State_New York', 'num__R&D Spend',
       'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[3]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Marketing Spend
col = ['cat__State_Florida', 'cat__State_New York', 'num__R&D Spend',
       'num__Administration']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[4]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

Si sacamos a cat__State_Florida, AIC nos da:
AIC 746.9
Si sacamos a cat__State_New York, AIC nos da:
AIC 747.0
Si sacamos a num__R&D Spend, AIC nos da:
AIC 820.5
Si sacamos a num__Administration, AIC nos da:
AIC 748.8
Si sacamos a num__Marketing Spend, AIC nos da:
AIC 748.7


De estos 5 modelos, el mejor es el que tiene eliminado `'cat__State_Florida'`, lo que reduce el AIC de `749` a `746.9`. Ahora, seguimos con el siguiente paso.

In [10]:
# cat__State_New York
col = ['num__R&D Spend', 'num__Administration', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[1]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__R&D Spend
col = ['cat__State_New York', 'num__Administration', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[2]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Administration
col = ['cat__State_New York', 'num__R&D Spend', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[3]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Marketing Spend
col = ['cat__State_New York', 'num__R&D Spend', 'num__Administration']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[4]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

Si sacamos a cat__State_New York, AIC nos da:
AIC 745.1
Si sacamos a num__R&D Spend, AIC nos da:
AIC 819.0
Si sacamos a num__Administration, AIC nos da:
AIC 746.8
Si sacamos a num__Marketing Spend, AIC nos da:
AIC 746.9


De estos 4 modelos, el mejor es el que tiene eliminado `'State_New_York'`, lo que reduce el AIC de `746.9` a `745.1`. Ahora, seguimos con el siguiente paso.

In [11]:
# num__R&D Spend
col = ['num__Administration', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[2]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Administration
col = ['num__R&D Spend', 'num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[3]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Marketing Spend
col = ['num__R&D Spend', 'num__Administration']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[4]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

Si sacamos a num__R&D Spend, AIC nos da:
AIC 817.1
Si sacamos a num__Administration, AIC nos da:
AIC 744.9
Si sacamos a num__Marketing Spend, AIC nos da:
AIC 745.2


Observamos que al quitar  `'Administration'`, el modelo mejora, pasando de un AIC de `745.1` a `744.9`. Ahora, veamos si podemos seguir eliminando atributos y verificar si un modelo con solo un atributo es mejor que el modelo con todos los atributos:

In [12]:
# num__R&D Spend
col = ['num__Marketing Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[2]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

# num__Marketing Spend
col = ['num__R&D Spend']
aic = train_reg_model(X_train_processed, y_train, col)
print(f"Si sacamos a {X_train_processed.columns[4]}, AIC nos da:")
print(f"AIC {np.round(aic, 1)}")

Si sacamos a num__R&D Spend, AIC nos da:
AIC 817.7
Si sacamos a num__Marketing Spend, AIC nos da:
AIC 746.7


Aquí observamos que:
- Al eliminar `'num__R&D Spend'`, el AIC es `817.7`, lo que es mayor que el AIC anterior de `745.1`.
- Al eliminar `'num__Marketing Spend'`, el AIC es `746.7`, que también es mayor que el AIC anterior de `745.1`.

Es decir, ninguno de estos casos mejora el modelo. Por lo tanto, el mejor modelo que encontramos es el que mantiene los atributos:

- `'num__R&D Spend'`
- `'num__Marketing Spend'`

Recordemos que estos atributos eran los que mostraban la mejor correlación con respecto a la variable objetivo (`Profit`).

---

## Entrenamiento y evaluación de modelo más simple

Con la selección realizada, ahora obtenemos el modelo, lo entrenamos y evaluamos con el set de evaluación.

**Nota**: Obsérvese que nunca usamos el dataset de evaluación para la selección de variables.

In [13]:
from sklearn.pipeline import Pipeline

# Nos quedamos solo con las columnas que nos interesan del dataset
numerical_columns_selected = ['R&D Spend', 'Marketing Spend']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns_selected)   
    ]
)

# Creamos el pipeline con preprocesamiento y modelo
pipeline_simple = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())  # Aplicamos la regresión lineal
])

# Entrenamos el modelo
pipeline_simple.fit(X_train, y_train)

# Resultados del entrenamiento:
print(f"El valor de la intersección de la recta es {np.round(pipeline_simple.named_steps['regressor'].intercept_, 2)}")
print(f"Los valores de los coeficientes de la recta son {np.round(pipeline_simple.named_steps['regressor'].coef_, 2)}")

print(f"El coeficiente de determinación (R^2) es {round(pipeline_simple.score(X_train, y_train), 2)}")

y_model = pipeline_simple.predict(X_train)
num_attributes = len(pipeline_simple.named_steps['preprocessor'].get_feature_names_out())

std_dev_model = np.sqrt((np.sum((y_train - y_model)**2))/(y_train.size - num_attributes - 1))
print(f"Desvío estándar del modelo {round(std_dev_model, 3)}")

El valor de la intersección de la recta es 111235.21
Los valores de los coeficientes de la recta son [36450.28  4483.09]
El coeficiente de determinación (R^2) es 0.95
Desvío estándar del modelo 9719.15


Ya podemos ver que el ajuste del modelo es prácticamente igual al que obtuvimos con todos los atributos.

Veamos ahora las métricas con respecto al set de evaluación:

In [14]:
from sklearn.metrics import (r2_score, mean_absolute_error, 
                             mean_squared_error, root_mean_squared_error, 
                             mean_absolute_percentage_error)

y_pred = pipeline_simple.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print("R-cuadrado en test:", round(r2, 3))
print("Error absoluto medio:", round(mae, 3))
print("Error cuadrático medio:", round(mse, 3))
print("Raíz de error cuadrático medio:", round(rmse, 3))
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")

R-cuadrado en test: 0.953
Error absoluto medio: 6449.238
Error cuadrático medio: 66537675.847
Raíz de error cuadrático medio: 8157.063
Error absoluto porcentual medio: 7.34%


Ahora comparemos el modelo con respecto al modelo de base y al modelo completo:

In [15]:
# Modelo baseline
mean_profit = np.mean(y_train)

y_pred_baseline = np.full_like(y_test, mean_profit)

r2_baseline = r2_score(y_test, y_pred_baseline)
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)
mse_baseline = mean_squared_error(y_test, y_pred_baseline)
rmse_baseline = root_mean_squared_error(y_test, y_pred_baseline)
mape_baseline = mean_absolute_percentage_error(y_test, y_pred_baseline)

print("R-cuadrado en test:", round(r2_baseline, 3))
print("Error absoluto medio:", round(mae_baseline, 3))
print("Error cuadrático medio:", round(mse_baseline, 3))
print("Raiz de error cuadrático medio:", round(rmse_baseline, 3))
print(f"Error absoluto porcentual medio: {mape_baseline*100:.2f}%")

R-cuadrado en test: -0.005
Error absoluto medio: 30301.288
Error cuadrático medio: 1413716446.334
Raiz de error cuadrático medio: 37599.421
Error absoluto porcentual medio: 35.79%


In [16]:
# Modelo con todos los atributos
import joblib

with open('reg_completo_50_startup.pkl', 'rb') as archivo:
    pipeline_complete = joblib.load(archivo)

y_pred_complete = pipeline_complete.predict(X_test)

r2 = r2_score(y_test, y_pred_complete)
mae = mean_absolute_error(y_test, y_pred_complete)
mse = mean_squared_error(y_test, y_pred_complete)
rmse = root_mean_squared_error(y_test, y_pred_complete)
mape = mean_absolute_percentage_error(y_test, y_pred_complete)

print("R-cuadrado en test:", round(r2, 3))
print("Error absoluto medio:", round(mae, 3))
print("Error cuadrático medio:", round(mse, 3))
print("Raíz de error cuadrático medio:", round(rmse, 3))
print(f"Error absoluto porcentual medio: {mape*100:.2f}%")

R-cuadrado en test: 0.94
Error absoluto medio: 7395.434
Error cuadrático medio: 84826955.035
Raíz de error cuadrático medio: 9210.155
Error absoluto porcentual medio: 8.93%


#### Comparación de modelos:

|                   | $R^2$      | MAE ($)   | RMSE ($) | MAPE (%) |
|-------------------|------------|-----------|----------|----------|
| Baseline          | -0.005     | 30301     | 3759     | 35.79    |
| Modelo completo   | 0.94       | 7395      | 9210     | 8.93     |
| **Modelo simple** | **0.953**  | **6449**  | **8157** | **7.34** |

En esta tabla se observa que, en este caso, menos atributos ajustan mejor con el set de evaluación. Obsérvese que esto no siempre ocurre al usar AIC, ya que este criterio solo nos asegura encontrar modelos más simples, pero puede que no siempre se obtengan métricas de regresión mejores. Esto se debe a que al reducir los atributos, sacrificamos parte de la información, pero en este caso, el modelo más simple ha logrado una mejor generalización y un ajuste superior.

Este modelo es mucho más simple, ya que utiliza solo dos atributos de la startup, lo que nos da una aproximación bastante precisa de sus ganancias anuales. Esto significa que, si el modelo se implementa en producción, será mucho más sencillo de mantener. No solo por la simplicidad en el número de atributos, sino también porque, al contar con un modelo más reducido, será menos costoso computacionalmente y más fácil de actualizar si se requieren cambios.

Por último, guardemos el modelo:

In [17]:
with open('reg_simple_50_startup.pkl', 'wb') as archivo:
    joblib.dump(pipeline_simple, archivo)